# Clone Voice

### Important:

This folder should be in relative path `OpenVoice/clone_voice.ipynb`
If the `/OpenVoice` directory is empty, then:

1. move this ipynb file out into `/2023QinOpenvoice` 
2. run the cell below
3. move this script into `/2023QinOpenvoice/OpenVoice`

In [ ]:
!git clone git@github.com:myshell-ai/OpenVoice.git

In [1]:
!cd OpenVoice && pip install -e .

Obtaining file:///home/avzh1/Documents/personal/kirill/lipsync/2023QinOpenvoice/OpenVoice
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Building editable for MyShell-OpenVoice (pyproject.toml) ... done
  Created wheel for MyShell-OpenVoice: filename=MyShell_OpenVoice-0.0.0-0.editable-py3-none-an

### Setup

In [ ]:
!wget https://myshell-public-repo-host.s3.amazonaws.com/openvoice/checkpoints_v2_0417.zip
!unzip checkpoints_v2_0417.zip -d OpenVoice/

In [ ]:
!pip install git+https://github.com/myshell-ai/MeloTTS.git
!python -m unidic download

### Inference

In [17]:
import os, sys

# Add the setup_data_vars function as we will need it to find the directory for the training data.
dir1 = os.path.abspath(os.path.join(os.path.abspath(''), 'OpenVoice'))
if not dir1 in sys.path: sys.path.append(dir1)

In [18]:
import torch

In [19]:
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

In [20]:
os.environ['LD_LIBRARY_PATH']

':/home/avzh1/Documents/personal/kirill/lipsync/.venv/lib/python3.10/site-packages/nvidia/cudnn/lib:/usr/local/cuda-8.0/lib64'

In [22]:
print('STARTING CELL EXECUTION')

ckpt_converter = '/home/avzh1/Documents/personal/kirill/lipsync/2023QinOpenvoice/OpenVoice/checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2_final'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

STARTING CELL EXECUTION
Loaded checkpoint '/home/avzh1/Documents/personal/kirill/lipsync/2023QinOpenvoice/OpenVoice/checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


If the below doesn't run because the kernel crashed, run the following in your command terminal:

`!find ~+/../../.venv/ -type f -name "libcudnn_ops_infer.so.8"`
and copy everything before "libcudnn_ops_infer.so.8" after the colon:
`!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<<<WHAT YOU'VE COPIED>>>`

In [23]:
reference_speaker = '/home/avzh1/Documents/personal/kirill/lipsync/content/reference_voice_cropped.mp3'

target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

OpenVoice version: v2


In [24]:
target_se.shape, audio_name

(torch.Size([1, 256, 1]), 'reference_voice_cropped_v2_VhdnIo3u0QF_^o+C0')

In [8]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/avzh1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
from melo.api import TTS

texts = {
    'EN_NEWEST': "Did you ever hear a folk tale about a giant turtle?",  # The newest English base speaker model
    'EN': "Did you ever hear a folk tale about a giant turtle?",
    'ES': "El resplandor del sol acaricia las olas, pintando el cielo con una paleta deslumbrante.",
    'FR': "La lueur dorée du soleil caresse les vagues, peignant le ciel d'une palette éblouissante.",
    'ZH': "在这次vacation中，我们计划去Paris欣赏埃菲尔铁塔和卢浮宫的美景。",
    'JP': "彼は毎朝ジョギングをして体を健康に保っています。",
    'KR': "안녕하세요! 오늘은 날씨가 정말 좋네요.",
}


src_path = f'{output_dir}/tmp.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)

# Speech to Speech

Intuitively, they use a basal text to speech model to obtain an audio of the text, to then
pass through the mimicking pipeline. It is possible to hijack this and provide your own
audio file to use.

In [25]:
# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    # audio_src_path=f'/home/avzh1/Documents/personal/kirill/lipsync/content/input_audio.wav', 
    # audio_src_path=f'/home/avzh1/Documents/personal/kirill/lipsync/content/input_vid.mp4', 
    audio_src_path = f'/home/avzh1/Documents/personal/kirill/lipsync/content/kirill_audio_original.wav',
    src_se=torch.load(f'/home/avzh1/Documents/personal/kirill/lipsync/2023QinOpenvoice/OpenVoice/checkpoints_v2/base_speakers/ses/en-default.pth', map_location=device), 
    tgt_se=target_se, 
    output_path=f'{output_dir}/output_v2_default_speech_to_speech.wav',
    message=encode_message)

# Text to Speech default

In [45]:
from melo.api import TTS

src_path = f'{output_dir}/default_customtext.wav'

# Speed is adjustable
speed = 0.9

text = """
Hello committee! ... I have decided to quickly dial in myself to endorse Kirill's application for the scholarship personally... I wanted to tell you three things about Kirill that made him stand out... Firstly, he's creative with technology... For example, I am not the real Steve Schwarzman, I am an Ey-eye model trained to mimic him. Artificial Intelligence is a transform-ative phenomenon, a mega trend. We need entrepreneurial people to deploy it. Secondly, he has lots of great stories from his travels. The one where he learned Mongolian throat singing while stranded in a yurt is my favorite! Finally, he's constantly learning! From pod-casts about Ancient Rome, to Singapore's remarkable urban infrastructure ... Kirill is always and always exploring. He's currently learning Mandarin, and his friend gave him a Chinese name - 聂珂儒 (nie ke ru). He would look forward to meeting you all.
"""

# text = "Testing the word: mega trend, Testing the word: Ey-eye, Testing the word Ay-eye"

language = "EN"

model = TTS(language=language, device=device)
speaker_ids = model.hps.data.spk2id

speaker_key = "EN-Default"
speaker_id = 4

speaker_key = speaker_key.lower().replace('_', '-')
    
source_se = torch.load(f'OpenVoice/checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
model.tts_to_file(text, speaker_id, src_path, speed=speed)
save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text split to sentences.
Hello committee! . . . I have decided to quickly dial in myself to endorse Kirill's application for the scholarship personally. . . I wanted to tell you three things about Kirill that made him stand out. . . Firstly, he's creative with technology. . . For example,
I am not the real Steve Schwarzman, I am an Ey-eye model trained to mimic him. Artificial Intelligence is a transform-ative phenomenon, a mega trend. We need entrepreneurial people to deploy it. Secondly, he has lots of great stories from his travels. The one where he learned Mongolian throat singing while stranded in a yurt is my favorite!
Finally, he's constantly learning! From pod-casts about Ancient Rome, to Singapore's remarkable urban infrastructure . . . Kirill is always and always exploring. He's currently learning Mandarin, and his friend gave him a Chinese name - 聂珂儒 nie ke ru. He would look forward to meeting you all.
 > ===========================


100%|██████████| 3/3 [00:00<00:00,  3.45it/s]


In [40]:
from melo.api import TTS

src_path = f'{output_dir}/default_customtext.wav'

# Speed is adjustable
speed = 1.0

text = """... 聂珂儒 ..."""
language = "ZH"

model = TTS(language=language, device=device)
speaker_ids = model.hps.data.spk2id

model = TTS(language=language, device=device)
speaker_ids = model.hps.data.spk2id

for speaker_key in speaker_ids.keys():
    speaker_id = speaker_ids[speaker_key]
    speaker_key = speaker_key.lower().replace('_', '-')
    
    source_se = torch.load(f'OpenVoice/checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
    model.tts_to_file(text, speaker_id, src_path, speed=speed)
    save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

    # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path, 
        src_se=source_se, 
        tgt_se=target_se, 
        output_path=save_path,
        message=encode_message)

/home/avzh1/Documents/personal/kirill/lipsync/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


 > Text split to sentences.
. . . 聂珂儒 . . .
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  8.06it/s]


Audio too short, fail to add watermark
